# SAM3 Fine-tuning for Hospital Floorplan Room Detection

This notebook fine-tunes SAM3 to detect room shapes in hospital floorplan images.

**Requirements:**
- Google Colab with A100 GPU
- Dataset in COCO format mounted on Google Drive

**Output:**
- Fine-tuned checkpoint for room segmentation
- TensorBoard logs for monitoring


## 1. Environment Setup


In [ ]:
# Check GPU
!nvidia-smi

import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# Clone SAM3 repository
!git clone https://github.com/facebookresearch/sam3.git
%cd sam3


In [ ]:
# Install SAM3 with training dependencies
!pip install -e ".[train]" -q

# Install additional dependencies
!pip install tensorboard opencv-python-headless -q


In [ ]:
# Authenticate with Hugging Face to download model weights
from huggingface_hub import login

# You need to request access at: https://huggingface.co/facebook/sam3
# Then create an access token at: https://huggingface.co/settings/tokens
login()  # This will prompt for your token


## 2. Dataset Configuration

Your dataset should be organized as:
```
drive/MyDrive/datasets/floorplans/
├── train/
│   ├── _annotations.coco.json
│   └── *.png (floorplan images)
├── val/
│   ├── _annotations.coco.json
│   └── *.png
└── test/
    ├── _annotations.coco.json
    └── *.png
```


In [ ]:
# Configuration - UPDATE THESE PATHS
DATASET_ROOT = "/content/drive/MyDrive/datasets/floorplans"
EXPERIMENT_DIR = "/content/drive/MyDrive/sam3_experiments/floorplans"
BPE_PATH = "assets/bpe_simple_vocab_16e6.txt.gz"

# Create experiment directory
import os
os.makedirs(EXPERIMENT_DIR, exist_ok=True)
os.makedirs(f"{EXPERIMENT_DIR}/checkpoints", exist_ok=True)
os.makedirs(f"{EXPERIMENT_DIR}/tensorboard", exist_ok=True)

print(f"Dataset root: {DATASET_ROOT}")
print(f"Experiment directory: {EXPERIMENT_DIR}")


In [ ]:
# Verify dataset exists
import json

train_ann_path = f"{DATASET_ROOT}/train/_annotations.coco.json"
val_ann_path = f"{DATASET_ROOT}/val/_annotations.coco.json"

def check_dataset(ann_path, split_name):
    if not os.path.exists(ann_path):
        print(f"❌ {split_name} annotations not found: {ann_path}")
        return False
    
    with open(ann_path, 'r') as f:
        data = json.load(f)
    
    num_images = len(data.get('images', []))
    num_annotations = len(data.get('annotations', []))
    has_segmentation = any('segmentation' in ann for ann in data.get('annotations', []))
    
    print(f"✓ {split_name}: {num_images} images, {num_annotations} annotations")
    if not has_segmentation:
        print(f"  ⚠️ WARNING: No segmentation masks found - mask training will fail!")
    return True

check_dataset(train_ann_path, "Train")
check_dataset(val_ann_path, "Validation")


## 3. Start Training

The training uses the pre-configured `floorplan_finetune.yaml` config which includes:
- Mask loss enabled for precise boundary detection
- A100-optimized batch sizes and learning rates
- 50 epochs with validation every 5 epochs


In [ ]:
# Start TensorBoard in the background
%load_ext tensorboard
%tensorboard --logdir {EXPERIMENT_DIR}/tensorboard


In [ ]:
# Run training
# Update paths in the config before running
!sed -i "s|floorplan_data_root:.*|floorplan_data_root: {DATASET_ROOT}|" sam3/train/configs/floorplans/floorplan_finetune.yaml
!sed -i "s|experiment_log_dir:.*|experiment_log_dir: {EXPERIMENT_DIR}|" sam3/train/configs/floorplans/floorplan_finetune.yaml

!python sam3/train/train.py \
    -c configs/floorplans/floorplan_finetune.yaml \
    --use-cluster 0 \
    --num-gpus 1


## 4. Monitor Training Progress


In [ ]:
# List checkpoints
import glob

checkpoints = glob.glob(f"{EXPERIMENT_DIR}/checkpoints/*.pt")
checkpoints.sort(key=os.path.getmtime, reverse=True)

print("Available checkpoints:")
for ckpt in checkpoints[:5]:
    size_mb = os.path.getsize(ckpt) / (1024 * 1024)
    print(f"  {os.path.basename(ckpt)} ({size_mb:.1f} MB)")


## 5. Test Inference with Fine-tuned Model


In [ ]:
import torch
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

from sam3.model_builder import build_sam3_image_model
from sam3.model.sam3_image_processor import Sam3Processor

# Load the fine-tuned model
# Update CHECKPOINT_PATH to your actual checkpoint
CHECKPOINT_PATH = f"{EXPERIMENT_DIR}/checkpoints/checkpoint_final.pt"

model = build_sam3_image_model(
    checkpoint_path=CHECKPOINT_PATH,
    bpe_path=BPE_PATH,
    enable_segmentation=True
)
processor = Sam3Processor(model)

print("✓ Model loaded successfully!")


In [ ]:
def visualize_rooms(image, masks, scores, threshold=0.5):
    """Visualize detected rooms on the floorplan."""
    plt.figure(figsize=(15, 15))
    plt.imshow(image)
    
    colors = plt.cm.tab20(np.linspace(0, 1, 20))
    
    room_count = 0
    for i, (mask, score) in enumerate(zip(masks, scores)):
        if score < threshold:
            continue
        
        room_count += 1
        color = colors[i % len(colors)]
        mask_overlay = np.zeros((*mask.shape, 4))
        mask_overlay[mask > 0] = [*color[:3], 0.4]
        plt.imshow(mask_overlay)
    
    plt.title(f"Detected {room_count} rooms (threshold={threshold})")
    plt.axis('off')
    plt.tight_layout()
    plt.show()

# Test on a sample image
test_images = glob.glob(f"{DATASET_ROOT}/test/*.png")

if test_images:
    test_image_path = test_images[0]
    image = Image.open(test_image_path)
    
    # Run inference
    inference_state = processor.set_image(image)
    output = processor.set_text_prompt(state=inference_state, prompt="room")
    
    masks = output["masks"]
    scores = output["scores"]
    
    print(f"Detected {len(masks)} potential rooms")
    print(f"Score range: {min(scores):.3f} - {max(scores):.3f}")
    
    visualize_rooms(image, masks, scores, threshold=0.5)
else:
    print("No test images found. Add images to the test folder.")


## 6. Export Model for Deployment


In [ ]:
# Copy the final checkpoint to a deployment-ready location
import shutil

final_checkpoint = f"{EXPERIMENT_DIR}/checkpoints/checkpoint_final.pt"
deployment_path = f"{EXPERIMENT_DIR}/sam3_floorplan_model.pt"

if os.path.exists(final_checkpoint):
    shutil.copy(final_checkpoint, deployment_path)
    print(f"✓ Model exported to: {deployment_path}")
    print(f"  Size: {os.path.getsize(deployment_path) / (1024*1024):.1f} MB")
else:
    # Try to find latest checkpoint
    if checkpoints:
        latest = checkpoints[0]
        shutil.copy(latest, deployment_path)
        print(f"✓ Latest checkpoint exported to: {deployment_path}")
    else:
        print("No checkpoints found. Run training first.")
